# **Normalized Difference Vegetation Index (NDVI) Dubai**

# Urban Evolution of Dubai - A Comparative Remote Sensing Analysis from 1995 to 2024

**Erik Ashkinadze (erik.ashkinadze@ruhr-uni-bochum.de)**

**Devon Klör (devon.kloer@ruhr-uni-bochum.de)**

**Course:** Geographic Information Systems (GIS I): Databases and Programming

**Professor:** Jun.-Prof. Dr. Andreas Rienow

**References:** Huang, S.; Tang, L.; Hupy, J.; Wang, Y.; Shao, G. (2020): A commentary review on the use of normalized difference vegetation index (NDVI) in the era of popular remote sensing. In: Journal of Forestry Research. 32, 1-6. DOI:  https://doi.org/10.1007/s11676-020-01155-1

Pettorelli, N.; Ryan, S.; Müller, T.; Bunnefeld, N.; Jedrzejewska, B.; Lima, M.; Kausrud, K. (2011): The Normalized Difference Vegetation Index (NDVI): unforeseen successes in animal ecology. In: CLIMATE RESEARCH, 46: 15-27.

**Repository:** GIS 1 Course "03_Create_landsat_timelapse_byQuishengWu"

In [ ]:
import ee
import geemap

# Authenticate and Initialize Earth Engine with the project ID
ee.Authenticate()
ee.Initialize(project='ee-dkloer01')

# Shapefile einladen (Pfad zur Datei anpassen)
shapefile_path = "./Dubai_Shapes/Dubai.shp"
dubai = geemap.shp_to_ee(shapefile_path)

# Loading the Image Collections from Landsat 5, 7 and 8
# Landsat 5 for 1995 (TM Sensor)
ls_ic_1995 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate('1995-01-01', '1995-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 7 for 2005 (TM Sensor)
ls_ic_2005 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterDate('2005-01-01', '2005-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2015 (OLI Sensor)
ls_ic_2015 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2015-01-01', '2015-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2025 (OLI Sensor)
ls_ic_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(dubai) \
    .median()

# Calculation of the Normalized Difference Vegetation Index (NDVI)
# Using band 4 (NIR) and band 5 (Red) for historic analysis with Landsat 5 & 7
# Using band 5 (NIR) and band 4 (Red) for current analysis with Landsat 8
def calculate_ndvi(image, year):
    if year <= 2005:
        # Landsat 5 & 7
        nir = image.select('SR_B4')
        red = image.select('SR_B3')
    else:
        # Landsat 8
        nir = image.select('SR_B5')
        red = image.select('SR_B4')

    # NDVI formula (orientated on Huang et al. 2020)
    numerator = (nir.subtract(red))
    denominator = (nir.add(red))
    ndvi = numerator.divide(denominator)
    return ndvi.addBands(ndvi.rename('NDVI'))

# Calculate NDVI for each year
ndvi_bands_1995 = calculate_ndvi(ls_ic_1995, 1995)
ndvi_1995 = ndvi_bands_1995.select('NDVI')

ndvi_bands_2005 = calculate_ndvi(ls_ic_2005, 2005)
ndvi_2005 = ndvi_bands_2005.select('NDVI')

ndvi_bands_2015 = calculate_ndvi(ls_ic_2015, 2015)
ndvi_2015 = ndvi_bands_2015.select('NDVI')

ndvi_bands_2024 = calculate_ndvi(ls_ic_2024, 2024)
ndvi_2024 = ndvi_bands_2024.select('NDVI')

# NDVI Change Detection (Difference between 1995 und 2024)
ndvi_change = ndvi_2024.subtract(ndvi_1995).rename('NDVI Change').clip(dubai)

# NDVI color palette for better differentiation of vegetation density
ndvi_palette = [
    '#800026',  
    '#BD0026', 
    '#E31A1C',  
    '#FC4E2A',  
    '#FEB24C',  
    '#C7E9B4',  
    '#41AB5D',  
    '#238443',  
    '#006837'   
]

# Color palette for NDVI change to emphasize changes over time
ndvi_change_palette = [
    '#d73027',  
    '#ffffbf',  
    '#1a9850'   
]

# Visualization settings for NDVI for better differentiation of vegetation states
ndvi_vis = {
    'min': -0.3, # The minimum NDVI value to be displayed in the visualization. Values below this will be colored with the lowest end of the palette.
    'max': 0.5, # The maximum NDVI value to be displayed. Values above this will be colored with the highest end of the palette.
    'palette': ndvi_palette
}

# Visualization settings for NDVI Change to highlight positive and negative changes
ndvi_change_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndvi_change_palette
}

# Create an interactive Map
map = geemap.Map(center=[25.07, 55.18], zoom=10)

# Add the Study Area as a Maplayer
map.addLayer(dubai.style(**{'color': 'black', 'width': 2}), {}, 'Study Area')

# Add the NDVI Layer to the Map
map.addLayer(ndvi_1995.clip(dubai), ndvi_vis, 'NDVI 1995')
map.addLayer(ndvi_2005.clip(dubai), ndvi_vis, 'NDVI 2005')
map.addLayer(ndvi_2015.clip(dubai), ndvi_vis, 'NDVI 2015')
map.addLayer(ndvi_2024.clip(dubai), ndvi_vis, 'NDVI 2024')
map.addLayer(ndvi_change.clip(dubai), ndvi_change_vis, 'NDVI Change')

# Display the Map
map


In [ ]:
years = [1995, 2005, 2015, 2024]
# It takes a list of 'ee.Image' objects as input and combines them into a single 'ee.ImageCollection' object.
ndvi_collection = ee.ImageCollection([ndvi_1995, ndvi_2005, ndvi_2015, ndvi_2024])

# The output of 'getInfo()' will be a Python dictionary containing metadata about the 'ndvi_collection'
print(ndvi_collection.getInfo())

In [ ]:
ndvi_change_ = ee.ImageCollection([ndvi_change])

In [ ]:
# NDVI color palette for better differentiation of vegetation density
ndvi_palette = [
    '#800026',  
    '#BD0026',  
    '#E31A1C',  
    '#FC4E2A', 
    '#FEB24C',  
    '#C7E9B4',  
    '#41AB5D',  
    '#238443',  
    '#006837'   
]

# Visualization settings for NDVI for better differentiation of vegetation states
ndvi_vis = {
    'min': -0.3,
    'max': 0.5, 
    'palette': ndvi_palette
}

In [ ]:
# Color palette for NDVI change to emphasize changes over time
ndvi_change_palette = [
    '#d73027',  
    '#ffffbf',  
    '#1a9850'   
]

# Visualization settings for NDVI Change to highlight positive and negative changes
ndvi_change_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndvi_change_palette
}


In [ ]:
# This dictionary 'video_args_ndvi' contains the necessary parameters to generate a time-lapse video of the NDVI data for Dubai.
video_args_ndvi = {
    'dimensions': 768,
    'region': dubai.geometry().bounds(),
    'framesPerSecond': 2,
    'min': ndvi_change_vis['min'],
    'max': ndvi_change_vis['max'],
    'palette': ndvi_change_vis['palette'],
    'format': 'gif'
}

In [ ]:
# specifies the output file path and name for the generated GIF video
out_gif_ndvi = './ndvi_selected_years.gif'
geemap.download_ee_video(ndvi_collection, video_args_ndvi, out_gif_ndvi) # This function is used to download a time-lapse video generated from a Google Earth Engine ImageCollection.

# This function takes an existing GIF video and adds text overlays to each frame.
geemap.add_text_to_gif(out_gif_ndvi, './ndvi_dubai_text.gif', text_sequence=years,
                       xy=('3%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

In [ ]:
# specifies the output file path and name for the generated GIF video
out_gif_ndvi_change = './ndvi_change.gif'
geemap.download_ee_video(ndvi_change_, video_args_ndvi, out_gif_ndvi_change) # This function is used to download a time-lapse video generated from a Google Earth Engine ImageCollection.

# This function takes an existing GIF video and adds text overlays to each frame.
geemap.add_text_to_gif(out_gif_ndvi_change, './ndvi_dubai_change.gif', text_sequence='Change Detection NDVI',
                       xy=('45%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

ndvi_palette = [
    '#800026',  
    '#BD0026',  
    '#E31A1C',  
    '#FC4E2A',  
    '#FEB24C',  
    '#C7E9B4',  
    '#41AB5D',  
    '#238443',  
    '#006837'   
]

# Creating a custom colormap from the list of colors
cmap_ndbsi = LinearSegmentedColormap.from_list("my_ba_cmap", ndvi_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ndbsi)

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Positions for the Ticks 
ax.set_xticklabels(['-1', '1'], color='white') # Labels for the Ticks


ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDVI.png", bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
ndvi_change_palette = [
    '#d73027',  
    '#ffffbf',  
    '#1a9850'   
]

# Creating a custom colormap from the list of colors
cmap_ndvi_change = LinearSegmentedColormap.from_list("my_ndvi_change_cmap", ndvi_change_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ndvi_change)

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Postions for the Ticks
ax.set_xticklabels(['decrease', 'increase'], color='white') # Labels for the Ticks

ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDVI_Change.png", bbox_inches='tight', pad_inches=0)
plt.close()